In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import emcee
%matplotlib inline

In [22]:
class node:
    def __init__(self,nInput,nNode,nOut):
        self.val=0
        self.lastVal=0
        self.init=0
        self.wIn=np.zeros(nInput)
        self.wNode=np.zeros((nNode-1,2))
        self.wOut=np.zeros(nOut)

def initNetwork(nInput,nNode,nOut,wNodes): #setting up the network
    nodes=[]
    for i in range(nNode):
        newNode=node(nInput,nNode,nOut)
        for j in range(nInput):
            newNode.wIn[j]=1 #for the moment treat all inputs as equally important
        for j in range(nOut):
            newNode.wOut[j]=1 #the same for outputs
        #and initialise all other weights at random
        for j in range(nNode-1):
            whichLink=j
            if j>=i:
                whichLink=j+1
            newNode.wNode[j,0]=whichLink
            newNode.wNode[j,1]=wNodes[i][j]
        nodes.append(newNode)
    return nodes

def sig(x):
    return 1/(1+np.exp(-x))

#nInput=2
#nNode=5
#nOut=1

#nodes=initNetwork(nInput,nNode,nOut)
#for i in range(nNode):
#    print(nodes[i].wNode)

In [23]:
def reward(weights,nNode,nInput):
    wNodes=np.split(weights,nNode)
    inputs=np.random.random(nInput)
    nodes=initNetwork(nInput,nNode,1,wNodes)
    nodes=resetNodes(nodes,inputs)
    trueOut=np.sum(inputs)
    netOut=runNetwork(nodes,100)
    diff=trueOut-netOut
    return np.exp(-(diff**2)/(2*trueOut**2))

def resetNodes(nodes,inputs):
    for i in range(nNode):
        nodes[i].lastVal=0
        nodes[i].init=0
        for j in range(nInput):
            nodes[i].init=nodes[i].init+nodes[i].wIn[j]*inputs[j]
    return nodes

def runNetwork(nodes,nMax):
    #grid=mpl.gridspec.GridSpec(nStep)
    lastOutput=0
    for step in range(nMax):
        for i in range(nNode): #setting value of nodes back to zero (plus constant contribution from input)
            nodes[i].val=nodes[i].init
        
        for i in range(nNode): #adding contribution from all other nodes
            thisNode=nodes[i]
            for j in range(nNode-1):
                nodes[int(thisNode.wNode[j,0])].val+=thisNode.lastVal*thisNode.wNode[j,1]
    
        norm=1
        #for i in range(nNode): #recording the normalised values for the next step
        #    norm=norm+nodes[i].val
    
        for i in range(nNode): #recording the normalised values for the next step
            nodes[i].lastVal=nodes[i].val/norm
        
        output=0
        #print("_______At step ",step,' nodes have values:')
        for i in range(nNode):
            #print(i,": ",nodes[i].lastVal)
            output+=nodes[i].lastVal*nodes[i].wOut[0]
        #print("giving output: ",output)
        
        if (np.abs((output-lastOutput)/output)<0.01):
            return output
        lastOutput=output
    return output

In [24]:
nNode=3
nInput=2
nWalkers=100
w0=[np.random.random(2*nNode) for i in range(nWalkers)]
sampler = emcee.EnsembleSampler(nWalkers, 2*nNode, reward, args=[nNode,nInput])
sampler.run_mcmc(w0,100)

/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: overflow encountered in double_scalars


NaN value of lnprob for parameters: 
[ -938.4138195   -281.02755825  1854.59957487 -1209.45378839  3635.09878384
  -986.70565607]
[ -875.63764763   -74.94990877  1107.84202695  -864.75626446  2234.47858869
   -29.31716134]


/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: overflow encountered in double_scalars
/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in double_scalars
/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: RuntimeWarning: overflow encountered in double_scalars
/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: RuntimeWarning: invalid value encountered in double_scalars


ValueError: lnprob returned NaN.

In [69]:
print(w0)

[array([ 0.29262455,  0.90027577,  0.94361137,  0.23583672,  0.17945005,
        0.80582912]), array([ 0.79985013,  0.37561112,  0.19352533,  0.14028728,  0.04980354,
        0.02629254]), array([ 0.03071264,  0.74173337,  0.08874342,  0.88235667,  0.80544967,
        0.78207465]), array([ 0.83118085,  0.24930534,  0.17712972,  0.07116731,  0.25538343,
        0.33478173]), array([ 0.10808455,  0.94278819,  0.82170849,  0.93862826,  0.6739173 ,
        0.10016786]), array([ 0.49265095,  0.59768931,  0.35974168,  0.73169921,  0.95381271,
        0.24612618]), array([ 0.7340929 ,  0.95004774,  0.72374514,  0.41762781,  0.50427481,
        0.37503184]), array([ 0.76349344,  0.58035805,  0.13420285,  0.59326866,  0.18205862,
        0.89899948]), array([ 0.06282772,  0.15547161,  0.29611484,  0.64253133,  0.57753692,
        0.18020146]), array([ 0.55427931,  0.596642  ,  0.49858871,  0.67468761,  0.62521527,
        0.30893084]), array([ 0.27694873,  0.61649331,  0.02270582,  0.04485602, 